In [3]:
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import cv2
import imageio
import imgaug as ia
from imgaug import augmenters as iaa
import numpy as np
%matplotlib inline
import xml.etree.ElementTree as ET
import pandas as pd 
import shutil 
import os 
import glob
from PIL import Image
from cv2 import imread

In [11]:
def read_content(xml_file: str):

    tree = ET.parse(xml_file)
    root = tree.getroot()

    list_with_all_boxes = []

    for boxes in root.iter('object'):

        filename = root.find('filename').text

        ymin, xmin, ymax, xmax = None, None, None, None

        ymin = int(float(boxes.find("bndbox/ymin").text))
        xmin = int(float(boxes.find("bndbox/xmin").text))
        ymax = int(float(boxes.find("bndbox/ymax").text))
        xmax = int(float(boxes.find("bndbox/xmax").text))

        list_with_single_boxes = [xmin, ymin, xmax, ymax, filename]
        list_with_all_boxes.append(list_with_single_boxes)

    return list_with_all_boxes

def cropncreateIMG(bnd_list):

    count = 0

    for boxes in bnd_list:

        count += 1

        xmin = boxes[0]
        ymin = boxes[1]
        xmax = boxes[2]
        ymax = boxes[3]

        filename = boxes[4]

        filename = filename.removeprefix('/Volumes/My Passport/originalData/') # **********

        filepath = "/Volumes/My Passport/CNNData/validation_set/" + filename 

        im = Image.open(filepath) 
        image = im.crop((xmin, ymin, xmax, ymax))
        image.save("/Volumes/My Passport/CNNData/validation_set/croppedImages/" + "num_" + str(count) + "_" + filename)

def backgroundncreate(bnd_list): 

    filename = bnd_list[0][4]
    filepath = "/Volumes/My Passport/CNNData/tick/" + filename 
    image = imread(filepath)

    for boxes in bnd_list:

        x = boxes[0]
        y = boxes[1]
        w = boxes[2]
        h = boxes[3]

        cv2.rectangle(image, (x, y), (w, h), (255, 255, 255), -1)

    cv2.imwrite("/Volumes/My Passport/CNNData/notick/paddedImages/"+filename, image)


In [13]:
folder = "/Volumes/My Passport/CNNData/validation_set"
for xmlfile in glob.iglob(os.path.join(folder, "*.xml")):
    if xmlfile != "/Volumes/My Passport/CNNData/validation_set/41227+Ixodes_scapularis+Discard+RTS_+life_unk+feed_unk.xml":
        boundingboxes = read_content(xmlfile)
        cropncreateIMG(boundingboxes)

In [48]:
folder = "/Volumes/My Passport/CNNData/tick"
for xmlfile in glob.iglob(os.path.join(folder, "*.xml")):
    boundingboxes = read_content(xmlfile)
    try: 
        backgroundncreate(boundingboxes)
    except:
        continue

In [16]:
def augment_jpg_xml(xmlfile_path, jpg_path): # goes based off of image path 
    boxes = read_content(xmlfile_path)
    image = imageio.imread(jpg_path)

    seq = iaa.Sequential(
        iaa.Rotate(-90))  


    boundbox1 = []
    bbs = BoundingBoxesOnImage(boundbox1, shape=image.shape)
    for list in boxes:
        boundbox1.append(BoundingBox(x1=list[0], x2=list[2], y1=list[1], y2=list[3]))


    # augments the bounding boxes 
    image_aug, bbs_aug = seq(image=image, bounding_boxes=bbs)

    df1 = pd.DataFrame(columns = ['after.x1', 'after.y1', 'after.x2', 'after.y2'])

    # puts all the bounding boxes in one xmlfile into a dataframe
    for i in range(len(bbs.bounding_boxes)):
        after = bbs_aug.bounding_boxes[i]
        df1 = df1.append({'after.x1' : after.x1, 'after.y1' : after.y1, 'after.x2' : after.x2, 'after.y2' : after.y2}, ignore_index = True)

    parsing = ET.parse(xmlfile_path)
    root1 = parsing.getroot()

    x = -1
    for xmin in root1.iter('xmin'):
        x = x+1
        new_xmin = df1.at[x,'after.x1']
        xmin.text = str(float(new_xmin))
              

    parsing.write(xmlfile_path)

    y = -1 
    for ymin in root1.iter('ymin'):
        y = y+1
        new_ymin = df1.at[y, 'after.y1']
        ymin.text = str(float(new_ymin))
        
 
    parsing.write(xmlfile_path)


    z = -1 
    for xmax in root1.iter('xmax'):
        z = z+1 
        new_xmax = df1.at[z, 'after.x2']
        xmax.text = str(float(new_xmax))


    parsing.write(xmlfile_path)


    e = -1 
    for ymax in root1.iter('ymax'):
        e = e+1 
        new_ymax = df1.at[e, 'after.y2']
        ymax.text = str(float(new_ymax))


    parsing.write(xmlfile_path)

In [21]:
augment_jpg_xml('/Volumes/My Passport/CNNData/tick/1031.xml', '/Volumes/My Passport/CNNData/tick/1031.jpg')